Под фильтрацией в DataFrame подразумевается получение новой таблицы путём вырезания строк, не удовлетворяющих поставленному условию. 

Разберём классический способ фильтрации в DataFrame — фильтрацию с помощью масок.

Маской называется Series, которая состоит из булевых значений, при этом значения True соответствуют тем индексам, для которых заданное условие выполняется, в противном случае ставится значение False (например, цена > 2 млн).

In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data.csv', sep = ',')

mask = melb_data['Price'] > 2000000
display(mask)

Для фильтрации нужно просто подставить переменную mask в индексацию DataFrame. Маска показывает, какие строки нужно оставлять в результирующем наборе, а какие — убирать (выведем первые пять строк отфильтрованной таблицы):

In [ ]:
display(melb_data[mask].head())

Примечание. В результате выполнения фильтрации возвращается новый DataFrame, полученный из исходного, при этом исходная таблица melb_data остаётся без изменений.

Также вовсе не обязательно заносить маску в отдельную переменную — можно сразу вставлять условие в операцию индексации DataFrame, например:

melb_data[melb_data['Price'] > 2000000]

Найдём количество зданий с тремя комнатами. Для этого отфильтруем таблицу по условию: обратимся к результирующей таблице по столбцу Rooms и найдём число строк в ней с помощью атрибута shape:

In [26]:
melb_data[melb_data['Rooms'] == 3].shape[0]

5881

Условия можно комбинировать, используя операторы & (логическое И) и | (логическое ИЛИ). Условия при этом заключаются в скобки.

In [27]:
melb_data[(melb_data['Rooms'] == 3) & (melb_data['Price'] < 300000)].shape[0]

3

Таких зданий оказалось всего три. Немного «ослабим» условие: теперь нас будут интересовать дома с ценой менее 300 тысяч, у которых либо число комнат равно 3 либо площадь домов более 100 квадратных метров:

In [ ]:
melb_data[((melb_data['Rooms'] == 3) | (melb_data['BuildingArea'] > 100)) & (melb_data['Price'] < 300000)].shape[0]
# 68

68

Примечание. Обратите внимание, что использование привычных операторов and и or будет неверным и приведёт к ошибке, так как они выполняют логические операции между двумя булевыми числами. В нашем случае слева и справа от оператора стоят маски (объекты Series), для которых логическую операцию надо совершить поэлементно, а операторы and и or для такого не предназначены.

Фильтрацию часто сочетают со статистическими методами. Давайте найдём максимальное количество комнат в таунхаусах. Так как в результате фильтрации получается DataFrame, то обратимся к нему по столбцу Rooms и найдём максимальное значение:

In [29]:
melb_data[melb_data['Type'] == 't']['Rooms'].max()
# 5

5

А теперь более сложный трюк: найдём медианную площадь здания у объектов, чья цена выше средней. Для того чтобы оградить наш код от нагромождений, предварительно создадим переменную со средней ценой:


In [30]:
mean_price = melb_data['Price'].mean()
melb_data[melb_data['Price'] > mean_price]['BuildingArea'].median()
# 126.0

126.0

Фильтрация находит применение в очистке данных.

Под очисткой данных понимается удаление из данных аномальных значений (выбросов), пропусков и данных, которые не несут информацию.

Причин появления таких дефектов в данных множество: неправильная передача по сети, неверная выгрузка из базы, человеческий фактор и т. д.

→ Проблема аномальных значений состоит в том, что если мы обучаем модель на данных, содержащих выбросы, то мы имеем немалый шанс получить менее верный прогноз, чего нам, конечно же, не хотелось бы.

In [ ]:
# У скольких объектов недвижимости из таблицы melb_data отсутствуют ванные комнаты?
melb_data[melb_data['Bathroom'] == 0].shape[0]

34

In [37]:
# Сколько в таблице melb_data объектов недвижимости, которые были проданы риелторской 
# компанией Nelson и стоимость которых составила больше 3 миллионов?

melb_data[(melb_data['SellerG'] == 'Nelson') & (melb_data['Price'] > 3000000)].shape[0]

5

In [41]:
#Какова минимальная стоимость участка без здания (площадь здания равна 0) в таблице melb_data? 
melb_data[melb_data['BuildingArea'] == 0]['Price'].min()

412500.0

In [55]:
# Какова средняя цена объектов недвижимости в таблице melb_data с ценой менее одного миллиона, 
# в которых либо количество комнат больше пяти, либо здание моложе 2015 года?

melb_data[(melb_data['Price'] < 1_000_000) & ((melb_data['Rooms'] > 5) | (melb_data ['YearBuilt'] > 2015))]['Price'].mean().round()

769239.0

In [57]:
# В каком районе Мельбурна чаще всего продаются 
# виллы и коттеджи (тип здания — h) с ценой меньше трёх миллионов?

melb_data[(melb_data['Type'] == 'h') & (melb_data['Price'] < 3_000_000)]['Regionname'].mode()

0    Northern Metropolitan
Name: Regionname, dtype: object